# Object Detection - YOLOS from Hugging Face

This model identifies objects present in images, returning its scores, labels and bounding boxes.


We are using a dataset from [UCF](https://www.crcv.ucf.edu/data/GMCP_Geolocalization/#Dataset) and the model [YOLOS](https://huggingface.co/hustvl/yolos-base) from [Hugging Face](https://huggingface.co/).

<a href="https://colab.research.google.com/github/VertaAI/examples/blob/main/deployment/huggingface/yolos-seq2seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Dependencies

This notebook has been tested with **Python 3.8.15** and the following package versions:

In [ ]:
%%capture
!pip install beautifulsoup4==4.6.3
!pip install requests==2.28.1
!pip install transformers==4.25.1
!pip install verta==0.21.1

## 2. Imports

In [ ]:
import concurrent.futures
import os
import pandas as pd
import requests
import time
import warnings

from bs4 import BeautifulSoup
from transformers import pipeline, YolosForObjectDetection, YolosFeatureExtractor
from verta import Client
from verta.endpoint.autoscaling import Autoscaling
from verta.endpoint.autoscaling.metrics import CpuUtilizationTarget, MemoryUtilizationTarget, RequestsPerWorkerTarget
from verta.endpoint.resources import Resources
from verta.environment import Python
from verta.registry import VertaModelBase, verify_io
from verta.utils import ModelAPI

warnings.simplefilter(action='ignore', category=FutureWarning)

## 3. Verta Set Up

In [ ]:
os.environ['VERTA_HOST'] = ''
os.environ['VERTA_EMAIL'] = ''
os.environ['VERTA_DEV_KEY'] = ''

In [ ]:
client = Client(os.environ['VERTA_HOST'])

## 4. Model Class

In [ ]:
class Yolos(VertaModelBase):
    def __init__(self, artifacts=None):
        self.weight = "tiny" # tiny/small/base
        self.model_name = f"hustvl/yolos-{self.weight}"
        self.model = pipeline(
            task = 'object-detection',
            model = YolosForObjectDetection.from_pretrained(self.model_name),
            feature_extractor = YolosFeatureExtractor.from_pretrained(self.model_name)
        )

    @verify_io
    def predict(self, urls):
        return self.model(urls) 

    def example(self):
        return ['http://www.cs.ucf.edu/~aroshan/index_files/Dataset_PitOrlManh/images/000001_1.jpg']

    def describe(self):
        return {
            'method': 'predict',
            'args': f"{self.example()}",
            'returns': """
                [
                    [{
                        'score': '',
                        'label': '',
                        'box': {
                            'xmin': '', 
                            'ymin': '', 
                            'xmax': '', 
                            'ymax': ''
                        }
                    }]
                ]
            """,
            'description': 'Identifies objects present in the images.',
            'input_description': 'List of image URLs.',
            'output_description': 'List with identified objects information.'
        }

## 5. Model Test

In [ ]:
yolos = Yolos()

In [ ]:
url = yolos.example()

In [ ]:
predictions = yolos.predict(url)

In [ ]:
results_rows = []

for item in predictions[0]:
    item = item.copy()
    item.update(item.pop('box'))
    results_rows.append(pd.Series(item))

In [ ]:
df = pd.concat(results_rows, axis=1).T

In [ ]:
df

## 6. Model Register

In [ ]:
registered_model = client.get_or_create_registered_model(name='YOLOS - Hugging Face')

In [ ]:
model = registered_model.create_standard_model(
      name = 'v0',
      model_cls = Yolos,
      model_api = ModelAPI([{'urls': yolos.example()}], yolos.predict(yolos.example())[0]),
      environment = Python(requirements = ['dill', 'Pillow', 'torch', 'transformers'])
)

## 7. Auto Scaling and Resources

In [ ]:
autoscaling = Autoscaling(min_replicas=1, max_replicas=20, min_scale=0.1, max_scale=10)
autoscaling.add_metric(CpuUtilizationTarget(0.6))
autoscaling.add_metric(MemoryUtilizationTarget(0.7))
autoscaling.add_metric(RequestsPerWorkerTarget(1))
resources = Resources(cpu=1., memory='4Gi')

## 8. Model Endpoint

In [ ]:
transformers_cache_env = {'TRANSFORMERS_CACHE': '/tmp'}

In [ ]:
endpoint = client.get_or_create_endpoint('yolos-hugging-face')

In [ ]:
%%time
endpoint.update(
    model,
    autoscaling = autoscaling,
    resources = resources,
    env_vars = transformers_cache_env,
    wait = True
)

In [ ]:
deployed_model = endpoint.get_deployed_model()

## 9. Predictions

In [ ]:
url = 'http://www.cs.ucf.edu/~aroshan/index_files/Dataset_PitOrlManh/images/'

In [ ]:
req = requests.get(url)

In [ ]:
soup = BeautifulSoup(req.text, 'lxml')

In [ ]:
urls = []

for link in soup.find_all('a'):
    href = link.get('href')
  
    if href.endswith('.jpg'):
        url = f"http://www.cs.ucf.edu/~aroshan/index_files/Dataset_PitOrlManh/images/{href}"
        urls.append(url)

In [ ]:
urls = urls[:10]

In [ ]:
def process_image(url):
    return deployed_model.predict(url)

In [ ]:
results = []
start_time = time.time()

with concurrent.futures.ThreadPoolExecutor() as executor:
    for url, result in zip(urls, executor.map(process_image, urls)):
        results.append([url, result])
        
end_time = time.time()

In [ ]:
total_time = end_time - start_time
total_time = time.strftime('%Hh %Mm %Ss', time.gmtime(total_time))

In [ ]:
print(f"Processing Time: {total_time} for {len(urls)} images.")

In [ ]:
results_rows = []

for img_result in results:
    url = img_result[0]
    entities = img_result[1]

    for item in entities:
        item = item.copy()
        item.update(item.pop('box'))
        item['url'] = url
        results_rows.append(pd.Series(item))

In [ ]:
df = pd.concat(results_rows, axis=1).T

In [ ]:
df

In [ ]:
endpoint.delete()